In [1]:
import logging
logging.basicConfig(filename='cipi.log', format='%(message)s', level=logging.INFO)

from listens import *
import keras
from preprocess import *
from chat_iface import *
from chat_proc import *


C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['affirmasi', 'broadcast', 'cctv_lewatmana', 'direktori', 'goodbye', 'google', 'jam', 'lihat_gambar', 'negasi', 'salam', 'tokopedia', 'translate', 'wikipedia', 'youtube']
from data.affirmasi import affirmasi
from data.broadcast import broadcast
from data.cctv_lewatmana import cctv_lewatmana
from data.direktori import direktori
from data.goodbye import goodbye
from data.google import google
from data.jam import jam
from data.lihat_gambar import lihat_gambar
from data.negasi import negasi
from data.salam import salam
from data.tokopedia import tokopedia
from data.translate import translate
from data.wikipedia import wikipedia
from data.youtube import youtube


In [2]:
def process():
    ear.stop()
    pass
    print("Ini prosesnya")

In [3]:
def transcrib():
    global speech
    speech = transcribe_speech(s_iface)
    if speech == None:
        speech = ""
    print("Ini Transkrip:",speech)

In [4]:
ear = Listen()
ear.set_process(process)
#ear.start(thres=800, process=process)

In [5]:
model = keras.models.load_model("./model/chippy_v1.model")

In [6]:
from selenium import webdriver

headless = False
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--use-fake-ui-for-media-stream")
if headless:
    chrome_options.add_argument("--headless")  
    
def start_iface():
    s_iface = webdriver.Chrome(chrome_options=chrome_options)
    s_iface.get('https://translate.google.com/?#id/id')
    assert "Google Translate" in s_iface.title
    talk("saya siap",s_iface)
    return s_iface
    
def mainbrowser(url):
    global m_iface
    m_iface = webdriver.Chrome(chrome_options=chrome_options)
    m_iface.get('https://translate.google.com/?#id/id')

In [7]:
s_iface = start_iface()

In [8]:
time.sleep(4)
debug=True

In [9]:
cipi_recognized=False
ear.debug = False
feature_dim_1 = 20
feature_dim_2 = 11
channel=1

while not cipi_recognized:
    print("Start waiting for hotword...")
    ear.start(thres=1200, timeout=0)
    #ear.write_to_file('test.wav')
    dataaudio=np.array(ear.audiodata).flatten()
    sample = au2mfcc(dataaudio)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    y_pred = model.predict(sample_reshaped)
    y_max = np.max(y_pred) #nilai akurasi prediksi
    ypred = np.argmax(y_pred) #yg di prediksi adalah cipi
    cipi_recognized = (y_max>0.9 and ypred==1)
    if cipi_recognized: 
        
        #RANDOMIZE this
        playdetected()
        
        print("Recognized - Predicted:" ,get_labels()[0][ypred],y_max, ypred)

        attention=True
        att_counter = 0
        while attention:
            rec_speech(s_iface)
            ear.debug = False
            ear.start(thres=800, timeout=5, process=transcrib)
            if speech != "": 
                
                #RANDOMIZE THIS
                playdetected()
                reply,humanstate,botstate = sequence(speech,humanstate,botstate)
                talk_until_complete(reply,s_iface)
            else:
                speech = " "
                att_counter += 1
                if att_counter > 1:
                    att_counter = 0
                    cipi_recognized=False
                    attention=False
                    
                    #RANDOMIZE THIS
                    playehm()
                    time.sleep(2)
                    print("\n=============\nGo to wait mode")

Start waiting for hotword...
Ini prosesnya
Start waiting for hotword...
Ini prosesnya
Recognized - Predicted: cipi 0.9993462 1


NameError: name 'speech' is not defined

Ini Transkrip: saya siap
